## Explore the dataset.

In [106]:
# Useful imports
import numpy as np
from fastai.vision.all import *

import librosa 
import librosa.display as display

In [107]:
# import dataset
files = get_files('../data/raw', recurse=True, folders=None)

## Reading from .wav files and converting to spectogram
Using scipy.io to read the .wav files and convert them to spectograms.

In [108]:
# unused functions
def plot_wave_function(file):
    data, sample_rate = librosa.load(file)
    display.waveshow(data)
def to_spectogram(file, sample_rate=-1):

    if sample_rate == -1:
        data, sr = librosa.load(file)
        DATA = librosa.stft(data)
        DATA_db = librosa.amplitude_to_db(abs(DATA))
    
        plt.figure(figsize=(14, 5))
        display.specshow(DATA_db, x_axis = 'time', y_axis = 'hz')
        plt.colorbar()
        
    else:
            data, sr = librosa.load(file, sr=sample_rate)
            DATA = librosa.stft(data)

            DATA_db = librosa.amplitude_to_db(abs(DATA))

            plt.figure(figsize=(14, 5))
            display.specshow(DATA_db, sr=sample_rate, x_axis = 'time', y_axis = 'hz')
            plt.colorbar()

In [109]:
to_spectogram(files[4], 8000)

In [110]:
to_spectogram(files[6], 8000)

In [111]:
to_spectogram(files[96], 8000)

In [112]:
to_spectogram(files[11], 8000)

In [118]:
# run the following 3 cells to convert .wav files to spectograms.
import os
import wave
import pylab

def graph_spectrogram(wav_file):
    sound_info, frame_rate = get_wav_info(wav_file)
    pylab.figure(num=None, figsize=(19, 12), frameon=False)
    pylab.subplot(111)
    pylab.title('spectrogram of %r' % wav_file)
    pylab.specgram(sound_info, Fs=frame_rate)
    outdir = wav_file.replace("raw", "interim").removesuffix('.wav')
    pylab.savefig(outdir + '.png')

def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.fromstring(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate


In [119]:
graph_spectrogram(f'{files[0]}')

/var/folders/lx/s13ydqvd3zz_zhxjtpp931cm0000gn/T/ipykernel_7527/2148225556.py:18: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  sound_info = pylab.fromstring(frames, 'int16')


In [ ]:
def graph_all_files(files):
    for file in files:
        graph_spectrogram(f'{file}')

In [ ]:
graph_all_files(files)

/var/folders/lx/s13ydqvd3zz_zhxjtpp931cm0000gn/T/ipykernel_7527/3445548921.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  sound_info = pylab.fromstring(frames, 'int16')


KeyboardInterrupt: 